<a href="https://colab.research.google.com/github/Djaxis/LA-PLATEFORME/blob/main/scraplaplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Résumé des tâches avec les technologies NLP utilisées et leur utilité

---

**Assurez-vous d'avoir téléchargé les ressources nécessaires.**

- **Utilité :** Télécharger les ressources nécessaires pour le traitement de texte avec NLTK, notamment les ponctuations et les mots vides (stopwords).

---

**Fonction : `get_all_links`**

- **Utilité :** Collecter tous les liens internes pour le crawling des pages.

- **Tâches :**
  - Récupérer tous les liens internes sur la page donnée.
  - Trouver toutes les balises `<a>` avec un attribut `href`.
  - Vérifier si le lien est relatif et le convertir en URL absolue.
  - Vérifier si le lien commence par l'URL de base.

- **Technologies NLP :** Non applicable (NA).

---

**Fonction : `clean_soup`**

- **Utilité :** Nettoyer le contenu HTML pour ne conserver que les informations pertinentes.

- **Tâches :**
  - Nettoyer la soup en enlevant les éléments indésirables.
  - Supprimer les balises `<script>` et `<style>`.
  - Supprimer les commentaires.
  - Supprimer les balises de publicité (exemple: `<div class="ad">`).
  - Supprimer les balises de navigation (exemple: `<nav>`).
  - Supprimer les balises de pied de page (exemple: `<footer>`).
  - Supprimer les balises de header (exemple: `<header>`).

- **Technologies NLP :** Non applicable (NA).

---

**Fonction : `generate_questions`**

- **Utilité :** Transformer des phrases en questions pour générer des paires question-réponse.

- **Tâches :**
  - Générer des questions à partir des phrases données.
  - Tokenizer la phrase en mots.
  - Filtrer les mots vides (stop words).
  - Reconstituer la phrase sans les mots vides.
  - Ajouter un point d'interrogation à la fin pour former une question.

- **Technologies NLP :**
  - `nltk.tokenize` (pour tokenizer les phrases en mots).
  - `nltk.corpus.stopwords` (pour filtrer les mots vides).
  - `nltk.tokenize.treebank.TreebankWordDetokenizer` (pour reconstituer les phrases).

---

**Fonction : `scrape_page`**

- **Utilité :** Extraire le contenu textuel des pages web et générer des paires question-réponse.

- **Tâches :**
  - Scraper les phrases de la page donnée.
  - Envoyer une requête GET à l'URL.
  - Vérifier si la requête a réussi (code 200).
  - Analyser le contenu HTML avec Beautiful Soup.
  - Nettoyer la soup.
  - Trouver tous les paragraphes (balises `<p>`).
  - Extraire le texte de chaque paragraphe et les retourner.
  - Générer des questions à partir des phrases.
  - Associer chaque question à sa phrase originale comme réponse.

- **Technologies NLP :**
  - Beautiful Soup (pour analyser le contenu HTML).
  - `generate_questions` (utilise les technologies NLP mentionnées ci-dessus).

---

**Fonction : `crawl_site`**

- **Utilité :** Automatiser le processus de scraping et de génération de questions-réponses pour toutes les pages d'un site web.

- **Tâches :**
  - Crawl toutes les pages du site à partir de l'URL de départ.
  - Ensemble des URLs déjà visitées.
  - Liste des URLs à visiter.
  - Dictionnaire pour stocker les questions-réponses par page.
  - Prendre la première URL de la liste.
  - Passer à l'URL suivante si elle a déjà été visitée.
  - Ajouter l'URL à l'ensemble des URLs visitées.
  - Scraper la page actuelle.
  - Ajouter les questions-réponses générées au dictionnaire.
  - Récupérer tous les liens internes de la page actuelle.
  - Ajouter les nouveaux liens à la liste des URLs à visiter.

- **Technologies NLP :**
  - Utilise `scrape_page` et ses technologies associées.

---

**Section principale du code**

- **Utilité :** Initialiser le processus de scraping et stocker les résultats.

- **Tâches :**
  - URL de la première page à scraper.
  - Scraper toutes les pages accessibles via les menus.
  - Enregistrer toutes les questions et réponses dans un fichier texte, classées par pages.

- **Technologies NLP :** Utilise `crawl_site` et ses technologies associées.

---

**Exécution finale**

- **Utilité :** Finaliser et organiser les résultats du scraping.

- **Tâches :**
  - Ajouter une ligne vide entre les pages.
  - Indiquer que le scraping est terminé.

- **Technologies NLP :** Non applicable (NA).

In [ ]:
! pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer

# Assurez-vous d'avoir téléchargé les ressources nécessaires
nltk.download('punkt')
nltk.download('stopwords')

def get_all_links(soup, base_url):
    """Récupérer tous les liens internes sur la page donnée."""
    links = []
    # Trouver toutes les balises <a> avec un attribut href
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Vérifier si le lien est relatif et le convertir en URL absolue
        if href.startswith('/'):
            full_url = urljoin(base_url, href)
            links.append(full_url)
        # Vérifier si le lien commence par l'URL de base
        elif href.startswith(base_url):
            links.append(href)
    return links

def clean_soup(soup):
    """Nettoyer la soup en enlevant les éléments indésirables."""
    # Supprimer les balises <script> et <style>
    for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()

    # Supprimer les commentaires
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # Supprimer les balises de publicité (exemple: <div class="ad">)
    for ad in soup.find_all(class_='ad'):
        ad.decompose()

    # Supprimer les balises de navigation (exemple: <nav>)
    for nav in soup.find_all('nav'):
        nav.decompose()

    # Supprimer les balises de pied de page (exemple: <footer>)
    for footer in soup.find_all('footer'):
        footer.decompose()

    # Supprimer les balises de header (exemple: <header>)
    for header in soup.find_all('header'):
        header.decompose()

    return soup

def generate_questions(phrases):
    """Générer des questions à partir des phrases données."""
    questions = []
    stop_words = set(stopwords.words('french'))

    for phrase in phrases:
        # Tokenizer la phrase en mots
        words = nltk.word_tokenize(phrase, language='french')

        # Filtrer les mots vides (stop words)
        filtered_words = [word for word in words if word.lower() not in stop_words]

        # Reconstituer la phrase sans les mots vides
        filtered_phrase = TreebankWordDetokenizer().detokenize(filtered_words)

        # Ajouter un point d'interrogation à la fin pour former une question
        question = f"{filtered_phrase} ?"
        questions.append(question)

    return questions

def scrape_page(url):
    """Scraper les phrases de la page donnée."""
    try:
        # Envoyer une requête GET à l'URL
        response = requests.get(url)
        # Vérifier si la requête a réussi (code 200)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f'La requête a échoué pour {url} avec l\'erreur : {e}')
        return [], None

    # Analyser le contenu HTML avec Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Nettoyer la soup
    soup = clean_soup(soup)

    # Trouver tous les paragraphes (balises <p>)
    paragraphs = soup.find_all('p')

    # Extraire le texte de chaque paragraphe et les retourner
    phrases = [paragraph.get_text() for paragraph in paragraphs]

    # Générer des questions à partir des phrases
    questions = generate_questions(phrases)

    # Associer chaque question à sa phrase originale comme réponse
    qa_pairs = list(zip(questions, phrases))

    return qa_pairs, soup

def crawl_site(start_url, base_url):
    """Crawl toutes les pages du site à partir de l'URL de départ."""
    visited = set()  # Ensemble des URLs déjà visitées
    to_visit = [start_url]  # Liste des URLs à visiter
    all_qa_pairs_by_page = {}  # Dictionnaire pour stocker les questions-réponses par page

    while to_visit:
        current_url = to_visit.pop(0)  # Prendre la première URL de la liste
        if current_url in visited:
            continue  # Passer à l'URL suivante si elle a déjà été visitée
        print(f'Scraping {current_url}')
        visited.add(current_url)  # Ajouter l'URL à l'ensemble des URLs visitées

        # Scraper la page actuelle
        qa_pairs, soup = scrape_page(current_url)
        all_qa_pairs_by_page[current_url] = qa_pairs  # Ajouter les questions-réponses générées au dictionnaire

        if soup:
            # Récupérer tous les liens internes de la page actuelle
            links = get_all_links(soup, base_url)
            for link in links:
                if link not in visited and link not in to_visit:
                    to_visit.append(link)  # Ajouter les nouveaux liens à la liste des URLs à visiter

    return all_qa_pairs_by_page

# URL de la première page à scraper
base_url = 'https://laplateforme.io'
start_url = base_url + '/qui-sommes-nous/'

# Scraper toutes les pages accessibles via les menus
all_qa_pairs_by_page = crawl_site(start_url, base_url)

# Enregistrer toutes les questions et réponses dans un fichier texte, classées par pages
with open('all_questions_answers.txt', 'w') as file:
    for page_url, qa_pairs in all_qa_pairs_by_page.items():
        file.write(f'Page URL: {page_url}\n')
        for question, answer in qa_pairs:
            file.write(f'Q: {question}\n')
            file.write(f'A: {answer}\n')
        file.write('\n')  # Ajouter une ligne vide entre les pages

print('Scraping terminé.')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\change\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\change\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scraping https://laplateforme.io/qui-sommes-nous/
Scraping https://laplateforme.io
Scraping https://laplateforme.io/journees-portes-ouvertes/
Scraping https://laplateforme.io/candidatures-cursus/
Scraping https://laplateforme.io/telechargement-brochure/
Scraping https://laplateforme.io/abonnement-newsletter/
Scraping https://laplateforme.io/bachelor-it/
Scraping https://laplateforme.io/master-of-science/
Scraping https://laplateforme.io/ai-school/centrale-digital-lab/
Scraping https://laplateforme.io/innovation-lab/
Scraping https://laplateforme.io/newsletter-ia-n1/
Scraping https://laplateforme.io/les-etudiants-de-la-plateforme_-remportent-la-regate-des-minots-2024/
Scraping https://laplateforme.io/business-solution-special-cybersecurite-hacking/
Scraping https://laplateforme.io/business-solution-special-martigues/
Scraping https://laplateforme.io/retrospective-business-solution-special-ia-data/
Scraping https://laplateforme.io/category/actualites/
Scraping https://laplateforme.io/cat

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping https://laplateforme.io/wp-content/uploads/2024/05/Imrane-Bendassi.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Scraping https://laplateforme.io/wp-content/uploads/2024/04/cv-Ines-Lorquet.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping https://laplateforme.io/wp-content/uploads/2024/04/cv-Helio-Aubrun.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping https://laplateforme.io/wp-content/uploads/2024/04/CV-Leo-Carrey.pdf
